In [1]:
import numpy as np
import pandas as pd
from os import listdir

def Breadth_of_Coverage(group):
    length = group.iloc[0]['SLen']
    cov = np.zeros(length)
    starts = group['SStart'].tolist()
    ends = group['SEnd'].tolist()
    for i in range(0, len(starts)):
        cov[starts[i]:ends[i]]=1
    return cov.sum()/length*100.0

def Avg_Depth_of_Coverage(group):
    length = group.iloc[0]['SLen']
    cov = np.zeros(length)
    starts = group['SStart'].tolist()
    ends = group['SEnd'].tolist()
    for i in range(0, len(starts)):
        cov[starts[i]:ends[i]]+=1
    return cov.mean()

In [2]:
data_dir = 'Filtered_O_Allele_Alignments.txt'
df = pd.read_csv(data_dir, sep = "\t")
df_grouped = df.groupby(['Sample', 'Subject'])

In [19]:
df_stats = pd.DataFrame()
df_stats['Breadth_Coverage'] = df_grouped.apply(Breadth_of_Coverage)
df_stats['Avg_Depth_Coverage'] = df_grouped.apply(Avg_Depth_of_Coverage)
df_stats['Num_Reads'] = df_grouped['Query'].agg(['count'])
df_stats[['Breadth_Coverage','Avg_Depth_Coverage']] = df_stats[['Breadth_Coverage','Avg_Depth_Coverage']].round(2)
df_stats = df_stats.reset_index()

In [20]:
df_stats

,Sample,Subject,Breadth_Coverage,Avg_Depth_Coverage,Num_Reads
0,Hotspr20SampleP4,Ga0374710_01_176_1198_O_allele1,98.73,262.34,1888
1,Hotspr20SampleP4,Ga0374715_43_3679_4227_O_allele1,68.50,80.75,312
2,Hotspr20SampleP4,Ga0374720_01_6981_8123_O_allele1,99.12,691.91,5525
3,Hotspr20SampleP4,Ga0374726_012_4764_5399_O_allele2,97.63,122.25,544
4,Hotspr20SampleP4,Ga0374729_16_2234_2812_O_allele1,76.74,449.86,1830
...,...,...,...,...,...
4101,HotsprottomLayer_2,Ga0393558_105_1946_2914_O_allele1,98.76,25.77,176
4102,HotsprottomLayer_2,Ga0393564_041_7289_8014_O_allele1,89.21,14.16,72
4103,HotsprottomLayer_2,Ga0393568_062_4825_5205_O_allele1,37.04,0.37,1
4104,HotsprottomLayer_2,Ga0393571_126_433_1125_O_allele1,65.94,16.47,80


In [21]:
T = df_stats.pivot_table(index = 'Subject', columns = 'Sample',
                         values = ['Breadth_Coverage', 'Avg_Depth_Coverage', 'Num_Reads'])
X = T.swaplevel(0,1,axis = 'columns')
cols = df_stats['Sample'].unique().tolist()
X = X.reindex(columns=X.columns.reindex(cols, level=0)[0])
X.columns.names = ('Sample', 'Stat')
X = X.fillna(0)

In [22]:
X.to_excel('O_Ortholog_Allele.xlsx')